In [21]:
import pandas as pd
from transformers import DistilBertForSequenceClassification, DistilBertTokenizer
import torch

In [22]:
data = pd.read_csv('training_data_english.csv')  # Replace with your CSV filename

In [23]:

tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')


In [24]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

# Load the model using PyTorch's native loading method
loaded_model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=len(data['label'].unique()))
model_state_dict_filename = "distilbert_sequence_classifier_state_dict_pt.pth"
loaded_model.load_state_dict(torch.load(model_state_dict_filename, map_location=device))
loaded_model.to(device)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


RuntimeError: Error(s) in loading state_dict for DistilBertForSequenceClassification:
	size mismatch for classifier.weight: copying a param with shape torch.Size([2, 768]) from checkpoint, the shape in current model is torch.Size([8, 768]).
	size mismatch for classifier.bias: copying a param with shape torch.Size([2]) from checkpoint, the shape in current model is torch.Size([8]).

In [ ]:
correct_predictions = 0
total_predictions = len(data)

for index, row in data.iterrows():
    user_input = row['text']
    original_label = row['label']

    user_input_encodings = tokenizer(user_input, truncation=True, padding=True, return_tensors='pt', max_length=512)
    user_input_ids = user_input_encodings['input_ids'].to(device)
    user_input_attention_mask = user_input_encodings['attention_mask'].to(device)

    with torch.no_grad():
        loaded_model.eval()
        outputs = loaded_model(input_ids=user_input_ids, attention_mask=user_input_attention_mask)
        predicted_label = torch.argmax(outputs.logits, dim=1).item()

    predicted_category = data['label'][predicted_label]

    if predicted_category == original_label:
        correct_predictions += 1

    # print(f'Text: {user_input}')
    print(f'Predicted Category: {predicted_category}')
    print(f'Original Category: {original_label}\n')

In [ ]:
accuracy = (correct_predictions / total_predictions) * 100
print(f'Accuracy: {accuracy:.2f}%')